# Tutorial 2: Building Agents

**Goal:** Create the complete 3-level HERON agent hierarchy (Field, Coordinator, System).

**Time:** ~20 minutes

---

## Agent Hierarchy

HERON uses a 3-level hierarchy where each level has different timing:

```
Level 3: SystemAgent (grid operator)      — tick_interval: 300s
    │
Level 2: CoordinatorAgent (microgrid)     — tick_interval: 60s
    │
Level 1: FieldAgent (battery, generator)  — tick_interval: 1s
```

**Why timing matters:**
- In real systems, devices update fast, coordinators update slower
- HERON captures this with `tick_config` on each agent
- Enables realistic event-driven testing (Tutorial 05)

## Step 1: Building a Field Agent (Battery)

Field agents control physical devices. The **essential HERON pattern** is two overrides:

```python
class MyFieldAgent(FieldAgent):
    def set_state(self):    # Define observable state
        self.state.features.append(MyFeature())
    
    def set_action(self):   # Define action space
        self.action.set_specs(dim_c=1, range=(...))
```

**Important:** Agents don't have `step()`. Physics/simulation lives in the **environment**.
- Agent: `observe()` → `act()` 
- Environment: runs physics → calls `agent.update_from_environment()`

In [ ]:
import numpy as np
from typing import Dict, Optional
from dataclasses import dataclass

from heron.agents.field_agent import FieldAgent
from heron.core.feature import FeatureProvider
from heron.scheduling.tick_config import TickConfig


# Feature from Tutorial 1 (simplified)
@dataclass
class BatterySOC(FeatureProvider):
    """Battery state of charge - visible to owner and coordinator."""
    visibility = ['owner', 'upper_level']
    soc: float = 0.5

    def vector(self) -> np.ndarray:
        return np.array([self.soc], dtype=np.float32)

    def names(self): return ['soc']
    def to_dict(self): return {'soc': self.soc}
    @classmethod
    def from_dict(cls, d): return cls(**d)
    def set_values(self, **kw):
        if 'soc' in kw: self.soc = np.clip(float(kw['soc']), 0.0, 1.0)

In [ ]:
class SimpleBattery(FieldAgent):
    """Battery agent - minimal HERON FieldAgent pattern.
    
    Action: power setpoint [-1, 1] where negative=discharge, positive=charge
    """
    
    def __init__(self, agent_id: str, initial_soc: float = 0.5, **kwargs):
        self.initial_soc = initial_soc
        super().__init__(agent_id=agent_id, **kwargs)
    
    # === ESSENTIAL HERON PATTERN ===
    
    def set_state(self):
        """Override: Add features to agent state."""
        self.soc_feature = BatterySOC(soc=self.initial_soc)
        self.state.features.append(self.soc_feature)
    
    def set_action(self):
        """Override: Define action space."""
        self.action.set_specs(dim_c=1, range=(np.array([-1.0]), np.array([1.0])))
    
    # === OPTIONAL: Handle environment feedback ===
    
    def update_state(self, **env_state):
        """Called by environment after physics simulation."""
        if 'soc' in env_state:
            self.soc_feature.set_values(soc=env_state['soc'])

In [ ]:
# Test SimpleBattery
battery = SimpleBattery(
    agent_id='bat_1',
    initial_soc=0.5,
    upstream_id='mg_1',
    tick_config=TickConfig.deterministic(tick_interval=1.0),
)

print(f"Agent: {battery.agent_id}")
print(f"Action space: {battery.action_space}")
print(f"Initial SOC: {battery.soc_feature.soc}")

# The HERON contract: observe() -> act()
obs = battery.observe()
print(f"\nObservation: {obs.local}")

# Act with upstream action (from coordinator)
battery.act(obs, upstream_action=np.array([-0.5]))
print(f"Action applied: {battery.action.vector()}")

# Environment simulates physics, then calls update_from_environment()
# (In real usage, the environment handles this)
battery.update_from_environment({'soc': 0.4})
print(f"SOC after env update: {battery.soc_feature.soc}")

## Step 2: Building a Generator Agent

Same pattern, different domain logic.

In [ ]:
@dataclass
class GenOutput(FeatureProvider):
    """Generator output - visible to owner, coordinator, and system."""
    visibility = ['owner', 'upper_level', 'system']
    p_mw: float = 0.0
    p_max: float = 5.0

    def vector(self) -> np.ndarray:
        return np.array([self.p_mw / self.p_max], dtype=np.float32)

    def names(self): return ['p_normalized']
    def to_dict(self): return {'p_mw': self.p_mw, 'p_max': self.p_max}
    @classmethod
    def from_dict(cls, d): return cls(**d)
    def set_values(self, **kw):
        if 'p_mw' in kw: self.p_mw = float(kw['p_mw'])


class SimpleGenerator(FieldAgent):
    """Generator agent - same HERON pattern as battery."""
    
    def __init__(self, agent_id: str, p_max: float = 5.0, **kwargs):
        self.p_max = p_max
        super().__init__(agent_id=agent_id, **kwargs)
    
    def set_state(self):
        self.output_feature = GenOutput(p_mw=0.0, p_max=self.p_max)
        self.state.features.append(self.output_feature)
    
    def set_action(self):
        self.action.set_specs(dim_c=1, range=(np.array([0.0]), np.array([1.0])))
    
    def update_state(self, **env_state):
        if 'p_mw' in env_state:
            self.output_feature.set_values(p_mw=env_state['p_mw'])


# Test
gen = SimpleGenerator('gen_1', p_max=10.0, tick_config=TickConfig.deterministic(1.0))
gen.act(gen.observe(), upstream_action=np.array([0.8]))
print(f"Generator action: {gen.action.vector()}")

## Step 3: Building a Coordinator Agent (Microgrid)

Coordinators manage field agents. The **essential pattern** is one override:

```python
class MyCoordinator(CoordinatorAgent):
    def _build_subordinates(self, configs, env_id=None, upstream_id=None):
        return {agent.agent_id: agent for agent in my_field_agents}
```

In [ ]:
from heron.agents.coordinator_agent import CoordinatorAgent
from heron.protocols.vertical import SetpointProtocol
from gymnasium.spaces import Box


class SimpleMicrogrid(CoordinatorAgent):
    """Microgrid coordinator - manages battery + generator."""
    
    def __init__(self, agent_id: str, **kwargs):
        self._my_id = agent_id  # Store for use in _build_subordinates
        super().__init__(
            agent_id=agent_id,
            protocol=SetpointProtocol(),
            tick_config=TickConfig.deterministic(tick_interval=60.0),
            **kwargs
        )
    
    # === ESSENTIAL HERON PATTERN ===
    
    def _build_subordinates(self, configs, env_id=None, upstream_id=None):
        """Override: Create and return subordinate agents."""
        self._battery = SimpleBattery(f'{self._my_id}_bat', upstream_id=self._my_id)
        self._gen = SimpleGenerator(f'{self._my_id}_gen', upstream_id=self._my_id)
        return {
            self._battery.agent_id: self._battery,
            self._gen.agent_id: self._gen,
        }

In [ ]:
# Test SimpleMicrogrid
mg = SimpleMicrogrid(agent_id='mg_1')

print(f"Microgrid: {mg.agent_id}")
print(f"Subordinates: {list(mg.subordinates.keys())}")

# Coordinator observes and acts
obs = mg.observe()
print(f"\nCoordinator observation keys: {obs.local.keys()}")

## Step 4: Building a System Agent (Grid Operator)

SystemAgent is the **top level (L3)**. It manages coordinators and tracks system-wide metrics.

**Essential pattern:** Same as FieldAgent (`set_state`, `set_action`), plus set `coordinators` dict.

```python
class MySystem(SystemAgent):
    def set_state(self):
        self.state.features.append(SystemMetric())
    
    # After init: self.coordinators = {id: coordinator}
```

In [ ]:
from heron.agents.system_agent import SystemAgent


@dataclass
class SystemFrequency(FeatureProvider):
    """Grid frequency - system-level metric."""
    visibility = ['system']
    frequency_hz: float = 60.0

    def vector(self) -> np.ndarray:
        return np.array([self.frequency_hz - 60.0], dtype=np.float32)  # Deviation

    def names(self): return ['freq_deviation']
    def to_dict(self): return {'frequency_hz': self.frequency_hz}
    @classmethod
    def from_dict(cls, d): return cls(**d)
    def set_values(self, **kw):
        if 'frequency_hz' in kw: self.frequency_hz = float(kw['frequency_hz'])


class SimpleGridSystem(SystemAgent):
    """System agent - manages multiple microgrids."""
    
    def __init__(self, agent_id: str, microgrids: list = None):
        self._init_mgs = microgrids or []
        super().__init__(
            agent_id=agent_id,
            tick_config=TickConfig.deterministic(tick_interval=300.0),
        )
        # Set up coordinators
        if microgrids:
            self.coordinators = {mg.agent_id: mg for mg in microgrids}
            for mg in microgrids:
                mg.upstream_id = agent_id
    
    # === ESSENTIAL HERON PATTERN ===
    
    def set_state(self):
        self.freq_feature = SystemFrequency()
        self.state.features.append(self.freq_feature)
    
    def set_action(self):
        pass  # System typically doesn't act directly


# Test
mg_0 = SimpleMicrogrid('mg_0')
mg_1 = SimpleMicrogrid('mg_1')
system = SimpleGridSystem('grid_system', microgrids=[mg_0, mg_1])

print(f"System: {system.agent_id}")
print(f"Coordinators: {list(system.coordinators.keys())}")
print(f"Frequency: {system.freq_feature.frequency_hz} Hz")

## Summary: The HERON Patterns

### FieldAgent Pattern
```python
class MyDevice(FieldAgent):
    def set_state(self):
        self.my_feature = MyFeature()
        self.state.features.append(self.my_feature)
    
    def set_action(self):
        self.action.set_specs(dim_c=1, range=(...))
    
    def update_state(self, **env_state):  # Optional: handle env feedback
        ...
```

### CoordinatorAgent Pattern
```python
class MyCoordinator(CoordinatorAgent):
    def _build_subordinates(self, configs, env_id=None, upstream_id=None):
        return {agent.agent_id: agent for agent in my_agents}
```

### SystemAgent Pattern
```python
class MySystem(SystemAgent):
    def set_state(self):
        self.state.features.append(SystemMetric())
    
    # After init: self.coordinators = {id: coordinator}
```

## Key Takeaways

| Agent Type | Level | Essential Override | Optional Override |
|------------|-------|-------------------|-------------------|
| `FieldAgent` | L1 | `set_state()`, `set_action()` | `update_state()` |
| `CoordinatorAgent` | L2 | `_build_subordinates()` | — |
| `SystemAgent` | L3 | `set_state()` | — |

**The HERON contract:**
- **Agents don't have `step()`** — physics lives in the environment
- `observe()` → `act(obs, upstream_action)` → environment runs → `update_from_environment()`
- Use `state.features.append()` — not `register_feature()`

**Two execution modes:**
- **Synchronous (training):** `observe()` → `act()`
- **Event-driven (testing):** `tick(scheduler, ...)` handles everything

---

**Next:** [03_building_environment.ipynb](03_building_environment.ipynb) — Create the environment (where physics lives)